10.1.1

In [2]:
import requests
import json

url = "https://api.weather.gov/gridpoints/OKX/34,36/forecast"
response = requests.get(url)
print(response)

<Response [200]>


In [2]:
forecast = response.json()
print(forecast.keys())

dict_keys(['@context', 'type', 'geometry', 'properties'])


In [3]:
print(forecast["properties"]["periods"])

[{'number': 1, 'name': 'Memorial Day', 'startTime': '2025-05-26T13:00:00-04:00', 'endTime': '2025-05-26T18:00:00-04:00', 'isDaytime': True, 'temperature': 73, 'temperatureUnit': 'F', 'temperatureTrend': '', 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 2}, 'windSpeed': '5 mph', 'windDirection': 'NE', 'icon': 'https://api.weather.gov/icons/land/day/sct?size=medium', 'shortForecast': 'Mostly Sunny', 'detailedForecast': 'Mostly sunny. High near 73, with temperatures falling to around 71 in the afternoon. Northeast wind around 5 mph.'}, {'number': 2, 'name': 'Tonight', 'startTime': '2025-05-26T18:00:00-04:00', 'endTime': '2025-05-27T06:00:00-04:00', 'isDaytime': False, 'temperature': 56, 'temperatureUnit': 'F', 'temperatureTrend': '', 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 2}, 'windSpeed': '3 mph', 'windDirection': 'SW', 'icon': 'https://api.weather.gov/icons/land/night/sct?size=medium', 'shortForecast': 'Partly Cloudy', 'detailedFor

In [4]:
import pandas as pd

forecast_df = pd.DataFrame(forecast["properties"]["periods"])
forecast_df[
    ["endTime", "isDaytime", "temperature", "windSpeed"]
].head()

,endTime,isDaytime,temperature,windSpeed
0,2025-05-26T18:00:00-04:00,True,73,5 mph
1,2025-05-27T06:00:00-04:00,False,56,3 mph
2,2025-05-27T18:00:00-04:00,True,74,2 to 9 mph
3,2025-05-28T06:00:00-04:00,False,57,3 to 9 mph
4,2025-05-28T18:00:00-04:00,True,65,3 to 10 mph


In [5]:
import re

def get_wind_speed(row):
    numbers = [int(num) for num in re.findall(r'\d+', row)]
    return max(numbers)

forecast_df["date"] = pd.to_datetime(forecast_df['endTime']).dt.date

forecast_df["wind_speed"] = forecast_df["windSpeed"].apply(
    get_wind_speed
)  # D

forecast_df[["date", "temperature", "wind_speed"]].head() 

,date,temperature,wind_speed
0,2025-05-26,73,5
1,2025-05-27,56,3
2,2025-05-27,74,9
3,2025-05-28,57,9
4,2025-05-28,65,10


In [6]:
daily_forecast = (
    forecast_df.groupby("date")
    .agg({
        "temperature": ["min", "max"],
        "wind_speed": "max"})
    .reset_index()
)

daily_forecast.columns = [
"date", "low_temp", "high_temp", "wind_speed"
]

daily_forecast.head() 

,date,low_temp,high_temp,wind_speed
0,2025-05-26,73,73,5
1,2025-05-27,56,74,9
2,2025-05-28,57,65,10
3,2025-05-29,58,72,10
4,2025-05-30,61,76,13


10.2.1

In [3]:
lat, lon = 40.7392, -73.9829

url = f"https://forecast.weather.gov/MapClick.php?lat={lat}&lon={lon}"
headers = {
    "User-Agent": "Paulo's data exploration"
}

response = requests.get(url, headers = headers)  # C
print(response)

<Response [200]>


In [4]:
print(response.text)

<!DOCTYPE html><html class="no-js">
    <head>
        <!-- Meta -->
        <meta name="viewport" content="width=device-width" />
        <link rel="schema.DC" href="http://purl.org/dc/elements/1.1/" />
        <title>National Weather Service</title>
        <meta name="DC.title" content="National Weather Service" />
        <meta name="DC.description" content="NOAA National Weather Service" />
        <meta name="DC.creator" content="US Department of Commerce, NOAA, National Weather Service" />
        <meta name="DC.date.created" scheme="ISO8601" content="2025-05-28T20:52:24+00:00" />
        <meta name="DC.language" scheme="DCTERMS.RFC1766" content="EN-US" />
        <meta name="DC.keywords" content="weather" />
        <meta name="DC.publisher" content="NOAA's National Weather Service" />
        <meta name="DC.contributor" content="National Weather Service" />
        <meta name="DC.rights" content="/disclaimer.php" />
        <meta name="rating" content="General" />
        <met

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <!-- Meta -->
  <meta content="width=device-width" name="viewport"/>
  <link href="http://purl.org/dc/elements/1.1/" rel="schema.DC"/>
  <title>
   National Weather Service
  </title>
  <meta content="National Weather Service" name="DC.title"/>
  <meta content="NOAA National Weather Service" name="DC.description"/>
  <meta content="US Department of Commerce, NOAA, National Weather Service" name="DC.creator"/>
  <meta content="2025-05-28T20:52:24+00:00" name="DC.date.created" scheme="ISO8601"/>
  <meta content="EN-US" name="DC.language" scheme="DCTERMS.RFC1766"/>
  <meta content="weather" name="DC.keywords"/>
  <meta content="NOAA's National Weather Service" name="DC.publisher"/>
  <meta content="National Weather Service" name="DC.contributor"/>
  <meta content="/disclaimer.php" name="DC.rights"/>
  <meta content="General" name="rating"/>
  <meta content="index,follow" name="robots"/>
  <!-- Icons -->
  <link href="/build/images/favicon.eab

In [7]:
forecast_items = soup.find_all("div", class_="tombstone-container")

for item in forecast_items:
    print(item.text, item.img)

This AfternoonHigh: 59 °FChance LightRain <img alt="This Afternoon: A 50 percent chance of light rain.  Cloudy, with a steady temperature around 59. Southeast wind around 8 mph. " class="forecast-icon" src="newimages/medium/ra50.png" title="This Afternoon: A 50 percent chance of light rain.  Cloudy, with a steady temperature around 59. Southeast wind around 8 mph. "/>
TonightLow: 57 °FRain <img alt="Tonight: Rain.  Low around 57. East wind 8 to 13 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. " class="forecast-icon" src="newimages/medium/nra100.png" title="Tonight: Rain.  Low around 57. East wind 8 to 13 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. "/>
ThursdayHigh: 73 °FRain Likelythen SlightChance Rain <img alt="Thursday: Rain likely, mainly before 11am.  Mostly cloudy, with a high near 73. Northeast wind 6 to 10 mph becoming north in the afternoon.  

In [9]:
import pandas as pd


forecast_data = []

for item in forecast_items:
    period = (
        item.find("p", class_="period-name")
        .get_text()
        .replace("Night", "")
    )
    temp = item.find("p", class_="temp").get_text()

    wind_text = item.find("img")["alt"]

    forecast_data.append(
        {
            "date": period,
            "temp": temp,
            "wind_speed": wind_text,
        }
    )

forecast_data = pd.DataFrame(forecast_data)
forecast_data.head()

,date,temp,wind_speed
0,This Afternoon,High: 59 °F,This Afternoon: A 50 percent chance of light r...
1,Tonight,Low: 57 °F,Tonight: Rain. Low around 57. East wind 8 to ...
2,Thursday,High: 73 °F,"Thursday: Rain likely, mainly before 11am. Mo..."
3,Thursday,Low: 62 °F,"Thursday Night: Mostly cloudy, with a low arou..."
4,Friday,High: 76 °F,"Friday: A 40 percent chance of showers, mainly..."


In [10]:
import re
forecast_data["temp"] = [
    int(re.search(r"\d+", t).group()) for t in forecast_data["temp"]
]
forecast_data.head() 

,date,temp,wind_speed
0,This Afternoon,59,This Afternoon: A 50 percent chance of light r...
1,Tonight,57,Tonight: Rain. Low around 57. East wind 8 to ...
2,Thursday,73,"Thursday: Rain likely, mainly before 11am. Mo..."
3,Thursday,62,"Thursday Night: Mostly cloudy, with a low arou..."
4,Friday,76,"Friday: A 40 percent chance of showers, mainly..."


In [11]:
for row in forecast_data["wind_speed"].head():
    print(row)

This Afternoon: A 50 percent chance of light rain.  Cloudy, with a steady temperature around 59. Southeast wind around 8 mph. 
Tonight: Rain.  Low around 57. East wind 8 to 13 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. 
Thursday: Rain likely, mainly before 11am.  Mostly cloudy, with a high near 73. Northeast wind 6 to 10 mph becoming north in the afternoon.  Chance of precipitation is 60%. New precipitation amounts of less than a tenth of an inch possible. 
Thursday Night: Mostly cloudy, with a low around 62. Southwest wind around 5 mph. 
Friday: A 40 percent chance of showers, mainly after 2pm.  Mostly cloudy, with a high near 76. Southwest wind 6 to 9 mph. 


In [12]:
forecast_data["wind_speed"] = [
    re.findall(r"\d+ mph", w) for w in forecast_data["wind_speed"]
]
forecast_data.head() 

,date,temp,wind_speed
0,This Afternoon,59,[8 mph]
1,Tonight,57,[13 mph]
2,Thursday,73,[10 mph]
3,Thursday,62,[5 mph]
4,Friday,76,[9 mph]


In [13]:
forecast_data["wind_speed"] = [
    int(row[0].replace(" mph", "")) if row else None 
    for row in forecast_data["wind_speed"]
]

forecast = (
    forecast_data.groupby("date", sort=False)
    .agg({"temp": ["min", "max"], "wind_speed": "max"})
    .reset_index()
)

forecast.columns = ["date", "low_temp", "high_temp", "wind_speed"]
forecast.head()

,date,low_temp,high_temp,wind_speed
0,This Afternoon,59,59,8.0
1,Tonight,57,57,13.0
2,Thursday,73,73,10.0
3,Thursday,62,62,5.0
4,Friday,76,76,9.0


10.3.3

In [14]:
from datetime import datetime, timedelta

now = datetime.now()
start = (now - timedelta(days=3)).strftime("%Y-%m-%d")
end = (now - timedelta(days=2)).strftime("%Y-%m-%d")

base = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"
query = "?$query="
select = "SELECT `created_date`, `unique_key` "
where = 'WHERE caseless_eq(`descriptor`, "Rat Sighting") '
dates = f"AND (`created_date` BETWEEN '{start}' :: floating_timestamp AND '{end}' :: floating_timestamp) "

full_url = base + query + select + where + dates

In [15]:
response = requests.get(full_url)
print(response.json()) 

[{'created_date': '2025-05-25T03:12:28.000', 'unique_key': '65061446'}, {'created_date': '2025-05-25T03:58:06.000', 'unique_key': '65055815'}, {'created_date': '2025-05-25T07:02:02.000', 'unique_key': '65063659'}, {'created_date': '2025-05-25T07:07:11.000', 'unique_key': '65055814'}, {'created_date': '2025-05-25T07:20:28.000', 'unique_key': '65059143'}, {'created_date': '2025-05-25T07:37:55.000', 'unique_key': '65060245'}, {'created_date': '2025-05-25T08:09:19.000', 'unique_key': '65063657'}, {'created_date': '2025-05-25T08:29:44.000', 'unique_key': '65058027'}, {'created_date': '2025-05-25T08:59:17.000', 'unique_key': '65060247'}, {'created_date': '2025-05-25T09:35:27.000', 'unique_key': '65058026'}, {'created_date': '2025-05-25T09:37:14.000', 'unique_key': '65056941'}, {'created_date': '2025-05-25T10:25:02.000', 'unique_key': '65060244'}, {'created_date': '2025-05-25T10:30:46.000', 'unique_key': '65062559'}, {'created_date': '2025-05-25T11:07:05.000', 'unique_key': '65061444'}, {'cre